<a href="https://colab.research.google.com/github/MS-H2020/Open/blob/main/main_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [オゾンレベルの分類](https://signate.jp/competitions/123)

## Google Colabの設定

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
#TEMP_DIR = "./temp"
INPUT_DIR:str = "/content/drive/MyDrive/Ozone_Level_Detection/01_input"
#TRAIN_DIR:str = INPUT_DIR + "/train_data"
#TEST_DIR:str = INPUT_DIR + "/test_data"
ANALYSIS_DIR:str = "/content/drive/MyDrive/Ozone_Level_Detection/03_code-analysis"
MODEL_DIR:str = "/content/drive/MyDrive/Ozone_Level_Detection//04_model"
SUBMISSION_DIR:str = '/content/drive/MyDrive/Ozone_Level_Detection/05_submission'
CHECKPOINT_PATH:str = MODEL_DIR

## メインコード

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
df = pd.read_csv(INPUT_DIR+"/train.tsv", index_col='Date', parse_dates=True, sep='\t')
df = df.drop(['id'], axis=1)

In [9]:
df

,WSR0,WSR1,WSR2,WSR3,WSR4,WSR5,WSR6,WSR7,WSR8,WSR9,...,RH50,U50,V50,HT50,KI,TT,SLP,SLP_,Precp,OZONE
Date,,,,,,,,,,,,,,,,,,,,,
1998-01-01,0.8,1.8,2.4,2.1,2.0,2.1,1.5,1.7,1.9,2.3,...,0.15,10.67,-1.56,5795.0,-12.10,17.90,10330.0,-55.0,0.00,0
1998-01-02,2.8,3.2,3.3,2.7,3.3,3.2,2.9,2.8,3.1,3.4,...,0.48,8.39,3.84,5805.0,14.05,29.00,10275.0,-55.0,0.00,0
1998-01-03,2.9,2.8,2.6,2.1,2.2,2.5,2.5,2.7,2.2,2.5,...,0.60,6.94,9.80,5790.0,17.90,41.30,10235.0,-40.0,0.00,0
1998-01-04,4.7,3.8,3.7,3.8,2.9,3.1,2.8,2.5,2.4,3.1,...,0.49,8.73,10.54,5775.0,31.15,51.70,10195.0,-40.0,2.08,0
1998-01-05,2.6,2.1,1.6,1.4,0.9,1.5,1.2,1.4,1.3,1.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.58,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001-06-29,0.8,0.9,0.3,0.3,0.7,0.8,1.3,1.9,3.0,3.0,...,0.45,8.65,-1.27,5870.0,27.80,44.80,10190.0,-25.0,0.05,0
2001-06-30,1.0,0.7,0.8,0.6,0.9,1.2,1.3,1.7,2.8,2.9,...,0.65,5.38,2.00,5850.0,35.60,49.60,10175.0,-15.0,0.18,0
2001-07-01,0.7,1.0,1.4,0.9,1.0,1.2,1.4,1.4,1.1,2.0,...,0.22,2.29,4.09,5855.0,32.10,48.20,10155.0,-20.0,0.23,0


In [10]:
# showing column wise %ge of NaN values they contains
null_col = []
for i in df.columns:
  print(i,"\t-\t", df[i].isna().mean()*100)
  if df[i].isna().mean()*100 > 0:
    null_col.append(i)

WSR0 	-	 2.131018153117601
WSR1 	-	 2.131018153117601
WSR2 	-	 2.209944751381215
WSR3 	-	 2.131018153117601
WSR4 	-	 2.131018153117601
WSR5 	-	 2.131018153117601
WSR6 	-	 2.0520915548539858
WSR7 	-	 1.8153117600631412
WSR8 	-	 1.8942383583267564
WSR9 	-	 2.0520915548539858
WSR10 	-	 2.0520915548539858
WSR11 	-	 1.973164956590371
WSR12 	-	 1.8942383583267564
WSR13 	-	 2.0520915548539858
WSR14 	-	 2.0520915548539858
WSR15 	-	 1.8942383583267564
WSR16 	-	 1.8942383583267564
WSR17 	-	 1.8153117600631412
WSR18 	-	 1.8942383583267564
WSR19 	-	 2.131018153117601
WSR20 	-	 2.131018153117601
WSR21 	-	 2.131018153117601
WSR22 	-	 2.131018153117601
WSR23 	-	 2.131018153117601
WSR_PK 	-	 1.5785319652722969
WSR_AV 	-	 1.5785319652722969
T0 	-	 0.3946329913180742
T1 	-	 0.3946329913180742
T2 	-	 0.4735595895816891
T3 	-	 0.3946329913180742
T4 	-	 0.3946329913180742
T5 	-	 0.3946329913180742
T6 	-	 0.4735595895816891
T7 	-	 0.31570639305445936
T8 	-	 0.31570639305445936
T9 	-	 0.4735595895816891
T10 

In [11]:
# Since dataset contains values in str format, changing it into float values

for i in df.columns[1:-1]:
  df[i] = df[i].astype(str).astype(float)

for i in null_col:
  df[i] = df[i].fillna(df[i].mean())

# lets check for null values again
for i in df.columns:
  print(i,"\t-\t", df[i].isna().mean()*100)

WSR0 	-	 0.0
WSR1 	-	 0.0
WSR2 	-	 0.0
WSR3 	-	 0.0
WSR4 	-	 0.0
WSR5 	-	 0.0
WSR6 	-	 0.0
WSR7 	-	 0.0
WSR8 	-	 0.0
WSR9 	-	 0.0
WSR10 	-	 0.0
WSR11 	-	 0.0
WSR12 	-	 0.0
WSR13 	-	 0.0
WSR14 	-	 0.0
WSR15 	-	 0.0
WSR16 	-	 0.0
WSR17 	-	 0.0
WSR18 	-	 0.0
WSR19 	-	 0.0
WSR20 	-	 0.0
WSR21 	-	 0.0
WSR22 	-	 0.0
WSR23 	-	 0.0
WSR_PK 	-	 0.0
WSR_AV 	-	 0.0
T0 	-	 0.0
T1 	-	 0.0
T2 	-	 0.0
T3 	-	 0.0
T4 	-	 0.0
T5 	-	 0.0
T6 	-	 0.0
T7 	-	 0.0
T8 	-	 0.0
T9 	-	 0.0
T10 	-	 0.0
T11 	-	 0.0
T12 	-	 0.0
T13 	-	 0.0
T14 	-	 0.0
T15 	-	 0.0
T16 	-	 0.0
T17 	-	 0.0
T18 	-	 0.0
T19 	-	 0.0
T20 	-	 0.0
T21 	-	 0.0
T22 	-	 0.0
T23 	-	 0.0
T_PK 	-	 0.0
T_AV 	-	 0.0
T85 	-	 0.0
RH85 	-	 0.0
U85 	-	 0.0
V85 	-	 0.0
HT85 	-	 0.0
T70 	-	 0.0
RH70 	-	 0.0
U70 	-	 0.0
V70 	-	 0.0
HT70 	-	 0.0
T50 	-	 0.0
RH50 	-	 0.0
U50 	-	 0.0
V50 	-	 0.0
HT50 	-	 0.0
KI 	-	 0.0
TT 	-	 0.0
SLP 	-	 0.0
SLP_ 	-	 0.0
Precp 	-	 0.0
OZONE 	-	 0.0


In [12]:
from imblearn.over_sampling import RandomOverSampler

oversample = RandomOverSampler()
x, y = oversample.fit_resample(df.drop(['OZONE'], axis=1), df['OZONE'])

df = pd.DataFrame(x, columns=df.drop(['OZONE'], axis=1).columns)
df['OZONE'] = y

df.head()

,WSR0,WSR1,WSR2,WSR3,WSR4,WSR5,WSR6,WSR7,WSR8,WSR9,...,RH50,U50,V50,HT50,KI,TT,SLP,SLP_,Precp,OZONE
0,0.8,1.8,2.4,2.1,2.0,2.1,1.5,1.7,1.9,2.3,...,0.150000,10.670000,-1.560000,5795.00000,-12.100000,17.900000,10330.000000,-55.000000,0.00,0
1,2.8,3.2,3.3,2.7,3.3,3.2,2.9,2.8,3.1,3.4,...,0.480000,8.390000,3.840000,5805.00000,14.050000,29.000000,10275.000000,-55.000000,0.00,0
2,2.9,2.8,2.6,2.1,2.2,2.5,2.5,2.7,2.2,2.5,...,0.600000,6.940000,9.800000,5790.00000,17.900000,41.300000,10235.000000,-40.000000,0.00,0
3,4.7,3.8,3.7,3.8,2.9,3.1,2.8,2.5,2.4,3.1,...,0.490000,8.730000,10.540000,5775.00000,31.150000,51.700000,10195.000000,-40.000000,2.08,0
4,2.6,2.1,1.6,1.4,0.9,1.5,1.2,1.4,1.3,1.4,...,0.281562,10.141031,0.456265,5815.91443,9.262858,37.227059,10164.373444,-0.431406,0.58,0


In [13]:
X = df.drop(['OZONE'], axis=1)
y = df['OZONE']

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X.head()

,WSR0,WSR1,WSR2,WSR3,WSR4,WSR5,WSR6,WSR7,WSR8,WSR9,...,T50,RH50,U50,V50,HT50,KI,TT,SLP,SLP_,Precp
0,-0.485694,0.413820,1.009917,0.749773,0.635094,0.723903,0.063856,-0.175578,-0.417577,-0.222557,...,-1.530948,-0.435762,0.329889,-0.052070,-0.565825,-1.149220,-1.890236,3.731471,-1.86971,-0.212299
1,1.210837,1.624009,1.814525,1.286835,1.790647,1.744092,1.355793,0.824145,0.636667,0.689571,...,-1.262655,1.090324,0.103396,0.779099,-0.432722,0.197165,-0.834779,2.548418,-1.86971,-0.212299
2,1.295664,1.278241,1.188719,0.749773,0.812872,1.094881,0.986668,0.733261,-0.154016,-0.056716,...,-1.638265,1.645265,-0.040645,1.696463,-0.632376,0.395390,0.334781,1.688016,-1.35194,-0.212299
3,2.822542,2.142662,2.172128,2.271448,1.435092,1.651347,1.263512,0.551493,0.021692,0.440809,...,-1.879728,1.136569,0.137172,1.810364,-0.832031,1.077592,1.323677,0.827614,-1.35194,1.894635
4,1.041184,0.673146,0.294710,0.123201,-0.342681,0.167437,-0.212987,-0.448230,-0.944699,-0.968844,...,-0.268196,0.172645,0.277342,0.258274,-0.287447,-0.049311,-0.052500,0.168835,0.01389,0.375212


In [15]:
import os
import time

import tensorflow
from tensorflow import keras
from tensorflow.keras import models, datasets, layers, applications, optimizers
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tensorflow.compat.v1.logging.set_verbosity(tensorflow.compat.v1.logging.ERROR)

In [16]:
!python --version
print(tensorflow.__version__)

Python 3.10.12
2.15.0


### One-Hold法